In [ ]:
import numpy as np
import pandas as pd
import cv2
from tensorflow import keras

# 加载训练数据集
train_df = pd.read_csv('/kaggle/input/machine-learning-in-science-ii-2023/training_norm.csv')
train_images = []
for image_id in train_df['image_id']:
image = cv2.imread(f'/kaggle/input/machine-learning-in-science-ii-2023/training_data/training_data/{image_id}.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
train_images.append(image)
train_images = np.array(train_images)
train_angles = np.clip(np.array(train_df['angle']), 0, 1)
train_speeds = np.clip(np.round(np.array(train_df['speed'])), 0, 1)

# 加载测试数据集
test_images = []
for i in range(1, 1021):
image = cv2.imread(f'/kaggle/input/machine-learning-in-science-ii-2023/test_data/test_data/{i}.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
test_images.append(image)
test_images = np.array(test_images)
test_ids = np.arange(1, 1021)

# 构建模型
image_input = keras.Input(shape=(240, 320, 3), name='image_input')
x = keras.layers.Conv2D(32, (3, 3), activation='relu')(image_input)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Flatten()(x)
image_output = keras.layers.Dense(256, activation='relu', name='image_output')(x)

speed_input = keras.Input(shape=(1,), name='speed_input')
angle_input = keras.Input(shape=(1,), name='angle_input')
angle_speed_concat = keras.layers.concatenate([angle_input, speed_input])
angle_speed_output = keras.layers.Dense(256, activation='relu', name='angle_speed_output')(angle_speed_concat)

x = keras.layers.concatenate([image_output, angle_speed_output])
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
output_angle = keras.layers.Dense(1, activation='sigmoid', name='angle_output')(x)
output_speed = keras.layers.Dense(1, activation='sigmoid', name='speed_output')(x)

model = keras.Model(inputs=[image_input, angle_input, speed_input], outputs=[output_angle, output_speed])

# 编译模型
model.compile(optimizer='adam', loss={'angle_output': 'mse', 'speed_output': 'mse'})

train_norm = pd.read_csv('/kaggle/input/machine-learning-in-science-ii-2023/training_norm.csv')

# 将angle和speed分别作为输出
train_angles = train_norm['angle'].values.reshape((-1, 1))
train_angles = np.clip(train_angles, 0, 1) # 将角度限制在0-1之间
train_speeds = train_norm['speed'].values.reshape((-1, 1))
train_speeds = np.round(train_speeds) # 将速度取整为0或1

# 训练模型
model.fit({'image_input': train_images, 'angle_input': train_angles, 'speed_input': train_speeds},
{'angle_output': train_angles, 'speed_output': train_speeds},
epochs=10, batch_size=32, verbose=1)

predictions_angle, predictions_speed = model.predict({'image_input': test_images, 'angle_input': np.zeros((1020, 1)), 'speed_input': np.zeros((1020, 1))})
predictions_speed = np.round(predictions_speed) # 将速度取整为0或1
predictions_angle = np.clip(predictions_angle, 0, 1) # 将角度限制在0-1之间

# 生成提交文件
submission_df = pd.DataFrame({'image_id': test_ids, 'angle': predictions_angle.reshape(-1), 'speed': predictions_speed.reshape(-1)})
submission_df.to_csv('submission.csv', index=False)

In [ ]:
!pip install opencv-python

import numpy as np
import pandas as pd
import cv2
from tensorflow import keras

# 加载训练数据集
train_df = pd.read_csv('/kaggle/input/machine-learning-in-science-ii-2023/training_norm.csv')
train_images = []
for image_id in train_df['image_id']:
    image = cv2.imread(f'/kaggle/input/machine-learning-in-science-ii-2023/training_data/training_data/{image_id}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    train_images.append(image)
    train_images = np.array(train_images)
train_angles = np.array(train_df['angle'])
train_speeds = np.array(train_df['speed'])

# 加载测试数据集
test_images = []
for i in range(1, 1021):
    image = cv2.imread(f'/kaggle/input/machine-learning-in-science-ii-2023/test_data/test_data/{i}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    test_images.append(image)
test_images = np.array(test_images)
test_ids = np.arange(1, 1021)

# 构建模型
image_input = keras.Input(shape=(240, 320, 3), name='image_input')
x = keras.layers.Conv2D(32, (3, 3), activation='relu')(image_input)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Flatten()(x)
image_output = keras.layers.Dense(256, activation='relu', name='image_output')(x)

speed_input = keras.Input(shape=(1,), name='speed_input')
angle_input = keras.Input(shape=(1,), name='angle_input')
angle_speed_concat = keras.layers.concatenate([angle_input, speed_input])
angle_speed_output = keras.layers.Dense(256, activation='relu', name='angle_speed_output')(angle_speed_concat)

x = keras.layers.concatenate([image_output, angle_speed_output])
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(2, activation='sigmoid', name='output')(x)

model = keras.Model(inputs=[image_input, angle_input, speed_input], outputs=[output])

# 编译模型
model.compile(optimizer='adam', loss='mse')

train_norm = pd.read_csv('/kaggle/input/machine-learning-in-science-ii-2023/training_norm.csv')
# 将angle和speed分别作为输出
train_angles = train_norm['angle'].values.reshape((-1, 1))
train_speeds = train_norm['speed'].values.reshape((-1, 1))

# 训练模型
model.fit({'image_input': train_images, 'angle_input': train_angles, 'speed_input': train_speeds},
{'angle_output': train_angles, 'speed_output': train_speeds},
epochs=10, batch_size=32, verbose=1)

predictions_angle, predictions_speed = model.predict({'image_input': test_images, 'angle_input': np.zeros((1020, 1)), 'speed_input': np.zeros((1020, 1))})
predictions_speed = np.round(predictions_speed) # 将速度取整为0或1
predictions_angle = np.clip(predictions_angle, 0, 1) # 将角度限制在0-1之间

# 生成提交文件
submission_df = pd.DataFrame({'image_id': test_ids, 'angle': predictions_angle.reshape(-1), 'speed': predictions_speed.reshape(-1)})
submission_df.to_csv('submission.csv', index=False)

In [1]:
import numpy as np
import pandas as pd
import cv2
from tensorflow import keras

# 加载训练数据集
train_df = pd.read_csv('/kaggle/input/machine-learning-in-science-ii-2023/training_norm.csv')
train_images = []
for image_id in train_df['image_id']:
    image = cv2.imread(f'/kaggle/input/machine-learning-in-science-ii-2023/training_data/training_data/{image_id}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    train_images.append(image)
train_images = np.array(train_images)
train_angles = np.array(train_df['angle'])
train_speeds = np.array(train_df['speed'])

# 加载测试数据集
test_images = []
for i in range(1, 1021):
    image = cv2.imread(f'/kaggle/input/machine-learning-in-science-ii-2023/test_data/test_data/{i}.png')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    test_images.append(image)
test_images = np.array(test_images)
test_ids = np.arange(1, 1021)

# 构建模型
image_input = keras.Input(shape=(240, 320, 3), name='image_input')
x = keras.layers.Conv2D(32, (3, 3), activation='relu')(image_input)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = keras.layers.Flatten()(x)
image_output = keras.layers.Dense(256, activation='relu', name='image_output')(x)

speed_input = keras.Input(shape=(1,), name='speed_input')
angle_input = keras.Input(shape=(1,), name='angle_input')
angle_speed_concat = keras.layers.concatenate([angle_input, speed_input])
angle_speed_output = keras.layers.Dense(256, activation='relu', name='angle_speed_output')(angle_speed_concat)

x = keras.layers.concatenate([image_output, angle_speed_output])
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(2, name='output')(x)

model = keras.Model(inputs=[image_input, angle_input, speed_input], outputs=[output])

# 编译模型
model.compile(optimizer='adam', loss={'output': 'mse'})

train_norm = pd.read_csv('/kaggle/input/machine-learning-in-science-ii-2023/training_norm.csv')
# 将angle和speed分别作为输出
train_angles = train_norm['angle'].values.reshape((-1, 1))
train_speeds = train_norm['speed'].values.reshape((-1, 1))

# 训练模型
model.fit({'image_input': train_images, 'angle_input': train_angles, 'speed_input': train_speeds},
          {'angle_output': train_angles, 'speed_output': train_speeds},
          epochs=10, batch_size=32, verbose=1)

predictions = model.predict({'image_input': test_images, 'angle_input': np.zeros((1020, 1)), 'speed_input': np.zeros((1020, 1))})

# 生成提交文件
submission_df = pd.DataFrame({'image_id': test_ids, 'angle': predictions[:, 0], 'speed': predictions[:, 1]})
submission_df.to_csv('submission.csv', index=False)


ImportError: libGL.so.1: cannot open shared object file: No such file or directory